# Examples to use Groupby, pivot.table in Pandas

In [1]:
# Import packages
from pathlib import Path
import numpy as np
import pandas as pd

# Load the data
Dataset sample of the Melbourn housing dataset available on Kaggle

In [2]:
dir_name = Path.cwd()
file = 'Data/melb_data.csv' # file name
# Read the csv.file
df = pd.read_csv(dir_name/file, usecols=['Price', 'Landsize', 'Distance', 'Type', 'Regionname']) # Select columns
# Filter data - Price < 3M AND Landsize < 1200
df = df[(df.Price<3_000_000) & (df.Landsize<1200)].reset_index(drop=True)
df.head()

,Type,Price,Distance,Landsize,Regionname
0,h,1480000.0,2.5,202.0,Northern Metropolitan
1,h,1035000.0,2.5,156.0,Northern Metropolitan
2,h,1465000.0,2.5,134.0,Northern Metropolitan
3,h,850000.0,2.5,94.0,Northern Metropolitan
4,h,1600000.0,2.5,120.0,Northern Metropolitan


## Basic groupby function
Perform the average distance for each category in df.Type (h, t, u)

In [3]:
df[['Type', 'Distance']].groupby('Type').mean() # First select cols = Type, Distance, second perform groupby 'Type'

,Distance
Type,
h,10.972498
t,9.892477
u,7.832750


Type h is further away from the central district (reference point)
***
## Customize the column name
Groupby function by default will not change the column name if we wish so. One way to accomplish this is by using the **agg function** instead of the mean function:

In [6]:
df[['Type', 'Distance']].groupby('Type').agg(
        avg_distance = ('Distance', 'mean')) # Agg allows to change the col name

,avg_distance
Type,
h,10.972498
t,9.892477
u,7.832750


Other way is by changing the column name afterwards, which is an *alternative* when we use the **agg function** to apply distinct funs to different columns ('Distance', 'mean'; 'Landsize', 'sum'; 'Regionname', 'min')
***
### Customized column names for agg multiple columns

In [8]:
df[['Type', 'Distance']].groupby('Type').agg(
        avg_distance = ('Distance', 'mean'),
        median_distance = ('Distance', 'median'),
        max_distance = ('Distance', 'max'),
        min_distance = ('Distance', 'min'))

,avg_distance,median_distance,max_distance,min_distance
Type,,,,
h,10.972498,10.5,47.3,1.2
t,9.892477,9.9,35.4,1.3
u,7.832750,7.3,41.0,0.0


In [9]:
# Other example
df[['Type', 'Distance', 'Landsize']].groupby('Type').agg(
        avg_distance = ('Distance', 'mean'),
        std_distance = ('Distance', 'std'),
        total_land = ('Landsize', 'sum'),
        avg_land = ('Landsize', 'mean'))

,avg_distance,std_distance,total_land,avg_land
Type,,,,
h,10.972498,6.026591,4613583.0,506.931436
t,9.892477,4.403079,234610.0,215.238532
u,7.832750,4.518480,403084.0,146.842987


## Using Lambda expressions
*Lambda* expression is a special form of functions in Python, and our main motivations are simplicity and practicality. The **agg function** accepts *lambda* expressions, so we can perform more complex calculations alongside the **groupby function**. <br>
Examples of using the *lambda* expression:
- Calculate the *mean price* for each type and convert it to millions

In [12]:
df[['Type', 'Price']].groupby('Type').agg(
        avg_price = ('Price', lambda x: x.mean()/1_000_000)).round(2) # Round 2 decimal for the agg.fun

,avg_price
Type,
h,1.18
t,0.93
u,0.60


## Work as Index parameter of nested DataFrame
In case of nested groupbs the resulting *nested* DataFrame does **not** look good

In [13]:
df[['Type', 'Regionname', 'Distance']].groupby(['Type', 'Regionname']).mean()

Distance
Type Regionname                           
h    Eastern Metropolitan        14.047737
     Eastern Victoria            34.140476
     Northern Metropolitan        8.921352
     Northern Victoria           30.690909
     South-Eastern Metropolitan  23.833952
     Southern Metropolitan        9.457646
     Western Metropolitan        10.460141
     Western Victoria            30.690000
t    Eastern Metropolitan        13.074138
     Northern Metropolitan        7.664765
     South-Eastern Metropolitan  21.588000
     Southern Metropolitan       10.292048
     Western Metropolitan         9.200000
u    Eastern Metropolitan        13.025581
     Eastern Victoria            40.333333
     Northern Metropolitan        5.917663
     South-Eastern Metropolitan  22.842857
     Southern Metropolitan        7.490284
     Western Metropolitan         8.800257

We got a multi-index to navigate in the *nested* DataFrame, which is **not** practical to have this as index. There is an optimal way, instead of *reset_index* function, as this example shows:

In [14]:
df[['Type', 'Regionname', 'Distance']].groupby(['Type', 'Regionname'], 
                                              as_index=False).mean()

,Type,Regionname,Distance
0,h,Eastern Metropolitan,14.047737
1,h,Eastern Victoria,34.140476
2,h,Northern Metropolitan,8.921352
3,h,Northern Victoria,30.690909
4,h,South-Eastern Metropolitan,23.833952
5,h,Southern Metropolitan,9.457646
6,h,Western Metropolitan,10.460141
7,h,Western Victoria,30.690000
8,t,Eastern Metropolitan,13.074138
9,t,Northern Metropolitan,7.664765


This reset the index instead of the previous option with multi-index assignment, which can be advantageous in certain examples
***
## Missing values
Using **groupby function** ignores the missing values, see the example:

In [20]:
# First assign NaN to have missing values
df.iloc[100:150, 0] = np.nan # iloc to navigate in sequence of index

# Let us calculate the avg_distance for each category 'Type'
df[['Type', 'Distance']].groupby('Type').mean()

,Distance
Type,
h,10.988913
t,9.891713
u,7.832139


We should have an extra *Type* that is NaN and the respective mean, but the **groupby function** ignores it. Alternative way to avoid this:

In [ ]:
df[['Type', 'Distance']].groupby('Type', dro)